### Part1

Create a Microservice that has end point process-Message

create a case Class Message ( </br>
     messageType string </br>
     message string </br>
     messageKey string (not be confused with partition key) </br>
)

process-Message should receive Message

There must be Three actors

        NetworkMessageProcessor that process the message of type NetworkMessage
        CloudMessageProcessor that process the message of type CloudMessage
        AppMessageProcessor that process the message of type AppMessage

        NetworkMessages are written to topic network-message
        CloudMessages are written to topic cloud-message
        AppMessages are written to topic app-message

 May be You think about creating an Actor Named MessageHandler
 to pass the message to the right actor
 








### Part2:

create another microservice
that creates three actors

   CloudListener
   NetworkListener
   AppListener

   Each of them should have an associated consumer correspong to the
   topic that stores the message

   no sooner any of these actor gets a message

   they need to pass the message to the 
   
   MessageGatherer Actor

   This actor has a kakfa producer that stores the received
   message to a topic name consilated-messages

In [ ]:
import akka.actor.{ActorRef, ActorSystem, Props}
import akka.kafka.Subscriptions
import akka.kafka.scaladsl.Consumer
import akka.stream.scaladsl.Sink
import spray.json._
import JsonFormats._
import akka.http.scaladsl.Http
import akka.http.scaladsl.model.StatusCodes
import akka.http.scaladsl.server.Directives._

object ConsumerMicroService {
  implicit val system: ActorSystem = ActorSystem("KafkaListenersSystem")

  def main(args: Array[String]): Unit = {

    val producer = KafkaProducerFactory.createProducer()
    val messageGatherer = system.actorOf(Props(new MessageGatherer(producer)), "messageGatherer")

    val cloudListener = system.actorOf(Props(new CloudListener(messageGatherer)), "cloudListener")
    val networkListener = system.actorOf(Props(new NetworkListener(messageGatherer)), "networkListener")
    val appListener = system.actorOf(Props(new AppListener(messageGatherer)), "appListener")

    val consumerSettings = KafkaConsumerFactory.consumerSettings(system)

    def startListener(topic: String, listener: ActorRef): Unit = {
      Consumer.plainSource(consumerSettings, Subscriptions.topics(topic))
        .map { record => record.value().parseJson.convertTo[Message] }
        .runWith(Sink.actorRef(listener, onCompleteMessage = "complete", onFailureMessage = (x: Throwable) => {
          new RuntimeException(s"There was an error: $x")
        }))
    }

    startListener("cloud-message", cloudListener)
    startListener("network-message", networkListener)
    startListener("app-message", appListener)

    println("Listeners are active and consuming messages...")

    val route = post {
      path("terminate") {
        system.terminate()
        complete(StatusCodes.OK, "System terminated")
      }
    }


    Http().newServerAt("0.0.0.0", 8081).bind(route)
    println("Server online at http://0.0.0.0:8081/")
  }
}